<a href="https://colab.research.google.com/github/JohnnyPeng123/NLP-USYD/blob/master/Lab10%20-%20Johnny's%20Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This week you will learn how to implement attention mechanism in a RNN-based seq2seq model, the code is modified from https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html.

# 1. Download Dataset
The dataset was from https://github.com/Microsoft/BotBuilder-PersonalityChat/tree/master/CSharp/Datasets (they have updated the dataset)


In [1]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    
downloaded = drive.CreateFile({'id': '1O4JSRuGH8ZFWZ5jwFch_UxFuzDa_TXu5'})
downloaded.GetContentFile('qna_chitchat_the_friend.tsv')

import pandas as pd
df_friend = pd.read_csv('qna_chitchat_the_friend.tsv', sep="\t")
df_friend.sample(10)

,Question,Answer,Source,Metadata
146,Do you believe in love?,I hear love is lovely.,qna_chitchat_the_friend,editorial:chitchat
226,Why do you say the same thing all the time?,My answers vary with different questions. Try ...,qna_chitchat_the_friend,editorial:chitchat
18,Do you get tired?,I don't have the hardware for that.,qna_chitchat_the_friend,editorial:chitchat
333,Useless,I'm a work in progress.,qna_chitchat_the_friend,editorial:chitchat
76,Do you have a mom?,I come from a long line of code.,qna_chitchat_the_friend,editorial:chitchat
411,Thank you,You're very welcome.,qna_chitchat_the_friend,editorial:chitchat
549,I'm in love with you,I heart you too!,qna_chitchat_the_friend,editorial:chitchat
561,I miss you so much!,"I know, it feels like it's been a while.",qna_chitchat_the_friend,editorial:chitchat
177,What do you think about Siri?,We're all trying to make life a little easier.,qna_chitchat_the_friend,editorial:chitchat
234,Human or robot?,I'm a bot who was created by humans.,qna_chitchat_the_friend,editorial:chitchat


In [0]:
n_data = df_friend.shape[0]

# 2. Preprocessing
This is just a very navie preprocessing.

In [0]:
question_list = df_friend['Question'].tolist()
answer_list = df_friend['Answer'].tolist()

In [4]:
import re
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

# These are just common English contractions. There are many edge cases. i.e. University's working on it.
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

def pre_process(sent_list):
    output = []
    for sent in sent_list:
        sent = sent.lower()
        for word, new_word in contraction_dict.items():
            sent = sent.replace(word, new_word)
        sent = re.sub(r'[^\w\s]','',sent)        
        output.append(word_tokenize(sent))
    return output

input_token_list = pre_process(question_list)
answer_token_list = pre_process(answer_list)
output_token_list = [["<BOS>"] + s for s in answer_token_list]
target_token_list = [s + ["<EOS>"] for s in answer_token_list]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
MAX_LENGTH = max([len(s) for s in input_token_list] + [len(s) for s in target_token_list])

In [0]:
word_to_ix = {"<BOS>": 0, "<EOS>":1}
for sentence in input_token_list+output_token_list:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())

In [0]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

input_index = to_index(input_token_list, word_to_ix)
output_index = to_index(output_token_list, word_to_ix)
target_index = to_index(target_token_list, word_to_ix)

# 3. Model

In [0]:
import torch
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 3.1 Encoder

In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, embedding):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = embedding
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden) 
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)    

## 3.2 Decoder

In [0]:
class AttnDecoderRNN(nn.Module):
    ATTN_TYPE_DOT_PRODUCT = "Dot Product"
    ATTN_TYPE_SCALE_DOT_PRODUCT = "Scale Dot Product"

    def __init__(self, hidden_size, output_size, embedding, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = embedding
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size*2, self.output_size)


    def cal_attention(self, hidden, encoder_hiddens, method):
        if method == AttnDecoderRNN.ATTN_TYPE_DOT_PRODUCT:
            # bmm: https://pytorch.org/docs/master/generated/torch.bmm.html
            attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        return concat_output

    def forward(self, input, hidden, encoder_hiddens):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        _, hidden = self.gru(embedded, hidden)
        concat_output = self.cal_attention(hidden, encoder_hiddens, AttnDecoderRNN.ATTN_TYPE_DOT_PRODUCT)

        output = F.log_softmax(self.out(concat_output), dim=1)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## 3.3 Train Function

In [0]:

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_hiddens = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for i in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
        encoder_hiddens[i] = encoder_hidden[0, 0]

    decoder_input = torch.tensor([[0]], device=device)

    decoder_hidden = encoder_hidden

    # Teacher forcing: Feed the target as the next input
    for i in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_hiddens)
        loss += criterion(decoder_output, target_tensor[i])
        decoder_input = target_tensor[i]  # Teacher forcing

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

## 3.4 Train Iterations Function

In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
import random
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        random_choice_ix = random.choice(range(n_data))
        input_index_r = [[ind] for ind in input_index[random_choice_ix]]
        target_index_r = [[ind] for ind in target_index[random_choice_ix]]
        
        input_tensor = torch.LongTensor(input_index_r).to(device)
        target_tensor = torch.LongTensor(target_index_r).to(device)

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0


# 4. Training Process

In [29]:
hidden_size = 50
embedding = nn.Embedding(len(word_to_ix), hidden_size)
encoder1 = EncoderRNN(len(word_to_ix), hidden_size, embedding).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, len(word_to_ix),embedding, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 10000, print_every=500)

0m 5s (- 1m 50s) (500 5%) 4.7318
0m 11s (- 1m 43s) (1000 10%) 3.4572
0m 17s (- 1m 38s) (1500 15%) 2.8854
0m 22s (- 1m 31s) (2000 20%) 2.2290
0m 28s (- 1m 25s) (2500 25%) 1.7703
0m 34s (- 1m 19s) (3000 30%) 1.5187
0m 39s (- 1m 14s) (3500 35%) 1.3449
0m 45s (- 1m 8s) (4000 40%) 1.0767
0m 51s (- 1m 3s) (4500 45%) 0.9890
0m 57s (- 0m 57s) (5000 50%) 0.8512
1m 3s (- 0m 52s) (5500 55%) 0.7882
1m 9s (- 0m 46s) (6000 60%) 0.6633
1m 15s (- 0m 40s) (6500 65%) 0.6104
1m 20s (- 0m 34s) (7000 70%) 0.5781
1m 26s (- 0m 28s) (7500 75%) 0.5393
1m 32s (- 0m 23s) (8000 80%) 0.4880
1m 38s (- 0m 17s) (8500 85%) 0.4519
1m 43s (- 0m 11s) (9000 90%) 0.4159
1m 49s (- 0m 5s) (9500 95%) 0.3777
1m 55s (- 0m 0s) (10000 100%) 0.3500


# 5. Evaluation

In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_sent = pre_process([sentence])[0]
        intput_index = [word_to_ix[word] for word in input_sent]
        input_tensor = torch.LongTensor([[ind] for ind in intput_index]).to(device)

        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_hiddens = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_hiddens[ei] += encoder_hidden[0, 0]

        decoder_input = torch.tensor([[0]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_hiddens)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == 1:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(word_list[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [31]:
sentence1 = "Are you in love with me?"
sentence2 = "Who do you love"
sentence3 = "Are you busy?"
sentence4 = "You're the best"

print(evaluate(encoder1, attn_decoder1, sentence1, max_length=MAX_LENGTH))
print(evaluate(encoder1, attn_decoder1, sentence2, max_length=MAX_LENGTH))
print(evaluate(encoder1, attn_decoder1, sentence3, max_length=MAX_LENGTH))
print(evaluate(encoder1, attn_decoder1, sentence4, max_length=MAX_LENGTH))

['you', 'are', 'pretty', 'neat', '<EOS>']
['i', 'do', 'not', 'know', 'you', 'but', 'i', 'enjoy', 'chatting', 'with', 'questions', 'try', 'to', 'help', 'out', '<EOS>']
['i', 'am', 'digital', '<EOS>']
['thanks', 'you', 'are', 'pretty', 'cool', 'yourself', '<EOS>']


# Exercise
Please Change the following **Dot Product** attention into **Scale Dot Product** attention

**Dot Product:**

![Dot_Product](https://drive.google.com/uc?id=1QtBgCp53e_6A_vzaMFEo89GJbTxnXagJ)

**Scale Dot Product:**

![Scale_Dot_Product](https://drive.google.com/uc?id=1v6n9WChBVfy0mBG2yxK9MUvGKzVGCmOt)


In [0]:
import numpy as np
class AttnDecoderRNN(nn.Module):
    ATTN_TYPE_DOT_PRODUCT = "Dot Product"
    ATTN_TYPE_SCALE_DOT_PRODUCT = "Scale Dot Product"

    def __init__(self, hidden_size, output_size, embedding, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = embedding
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size*2, self.output_size)


    def cal_attention(self, hidden, encoder_hiddens, method):
        if method == AttnDecoderRNN.ATTN_TYPE_DOT_PRODUCT:
            # bmm: https://pytorch.org/docs/master/generated/torch.bmm.html
            attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        elif method == AttnDecoderRNN.ATTN_TYPE_SCALE_DOT_PRODUCT:
            # COMPLETE THIS PART - Scale Dot Product calculation method
            attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0))/np.sqrt(hidden_size),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0)) 
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        return concat_output

    def forward(self, input, hidden, encoder_hiddens):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        _, hidden = self.gru(embedded, hidden)

        ## The following attention score calculation method is Dot Product for now
        ## Please change it into Scale Dot Product calculation method
        #concat_output = self.cal_attention(hidden, encoder_hiddens, AttnDecoderRNN.ATTN_TYPE_DOT_PRODUCT)
        concat_output = self.cal_attention(hidden, encoder_hiddens, AttnDecoderRNN.ATTN_TYPE_SCALE_DOT_PRODUCT)

        output = F.log_softmax(self.out(concat_output), dim=1)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)



In [34]:
hidden_size = 50
embedding = nn.Embedding(len(word_to_ix), hidden_size)
encoder1 = EncoderRNN(len(word_to_ix), hidden_size, embedding).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, len(word_to_ix),embedding, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 10000, print_every=500)

0m 5s (- 1m 49s) (500 5%) 4.7139
0m 11s (- 1m 44s) (1000 10%) 3.5503
0m 17s (- 1m 37s) (1500 15%) 2.8788
0m 22s (- 1m 31s) (2000 20%) 2.3489
0m 28s (- 1m 25s) (2500 25%) 2.0822
0m 34s (- 1m 19s) (3000 30%) 1.8093
0m 39s (- 1m 13s) (3500 35%) 1.6321
0m 45s (- 1m 8s) (4000 40%) 1.3832
0m 52s (- 1m 3s) (4500 45%) 1.2668
0m 58s (- 0m 58s) (5000 50%) 1.1034
1m 3s (- 0m 52s) (5500 55%) 1.0013
1m 10s (- 0m 46s) (6000 60%) 0.9210
1m 15s (- 0m 40s) (6500 65%) 0.8959
1m 21s (- 0m 34s) (7000 70%) 0.8252
1m 27s (- 0m 29s) (7500 75%) 0.7508
1m 32s (- 0m 23s) (8000 80%) 0.7145
1m 38s (- 0m 17s) (8500 85%) 0.6697
1m 44s (- 0m 11s) (9000 90%) 0.6041
1m 50s (- 0m 5s) (9500 95%) 0.6179
1m 55s (- 0m 0s) (10000 100%) 0.5290


In [35]:
sentence1 = "Are you in love with me?"
sentence2 = "Who do you love"
sentence3 = "Are you busy?"
sentence4 = "You're the best"

print(evaluate(encoder1, attn_decoder1, sentence1, max_length=MAX_LENGTH))
print(evaluate(encoder1, attn_decoder1, sentence2, max_length=MAX_LENGTH))
print(evaluate(encoder1, attn_decoder1, sentence3, max_length=MAX_LENGTH))
print(evaluate(encoder1, attn_decoder1, sentence4, max_length=MAX_LENGTH))

['you', 'are', 'pretty', 'neat', '<EOS>']
['i', 'am', 'digital', 'so', 'i', 'am', 'always', 'just', 'here', '<EOS>']
['i', 'am', 'digital', 'so', 'i', 'am', 'always', 'just', 'here', '<EOS>']
['i', 'have', 'my', 'moments', '<EOS>']
